# Natural Language Processing
## Assignment 2: Training NMT model with fairseq

### 1. Introduction

- 본 과제의 목적은 대표적인 pytorch library 중 하나인 fairseq을 이용해 번역 모델을 학습하는 방법을 배우는 것입니다.
- 일반적으로 우리는 해당 task와 관련되어 널리 알려진 library를 이용해 모델을 구현하고 tuning하게 됩니다. 자연어 처리 관련 여러 라이브러리가 있지만 번역 task에서 가장 자주 활용되고 있는 fairseq을 소개해드리고자 합니다. fairseq은 pytorch를 개발하고 있는 facebook에서 작업 중인 오픈소스 프로젝트 입니다. library의 이름처럼 sequence를 다루는 데에 필요한 여러 모델과 전처리, 평가 관련 코드를 포함해 인기가 많은 library 중 하나입니다.
- 이번 시간에는 해당 library의 [docs](https://fairseq.readthedocs.io/en/latest/)를 직접 읽어보면서 목표 perplexity/BLEU를 달성하는 것이 목표입니다. 프로젝트에 대한 대략적인 설명과 관련 docs 링크를 함께 제공해드리겠습니다. 주어진 데이터에 대해 **BLEU score 25 이상**을 달성해보세요 !
- ***먼저 colab 상단 탭 runtime -> change runtime type에서 hardware accelerator를 GPU로 변경해주세요***

In [1]:
# 필요 패키지 설치 (참고: 느낌표를 앞에 붙이면 해당 코드가 terminal에서 실행됩니다.)
!pip3 install fastBPE sacremoses subword_nmt hydra-core omegaconf fairseq

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\songbae\anaconda3\python.exe' 'C:\Users\songbae\anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' get_requires_for_build_wheel 'C:\Users\songbae\AppData\Local\Temp\tmpk9kgl96y'
       cwd: C:\Users\songbae\AppData\Local\Temp\pip-install-36p1gwa9\fairseq
  Complete output (31 lines):
  Traceback (most recent call last):
    File "setup.py", line 214, in <module>
      do_setup(package_data)
    File "setup.py", line 136, in do_setup
      setup(
    File "C:\Users\songbae\AppData\Local\Temp\pip-build-env-metig_ed\overlay\Lib\site-packages\setuptools\__init__.py", line 152, in setup
      _install_setup_requires(attrs)
    File "C:\Users\songbae\AppData\Local\Temp\pip-build-env-metig_ed\overl

In [ ]:
# clone fairseq git
!git clone https://github.com/pytorch/fairseq.git
# iwslt14 데이터 준비 (https://github.com/pytorch/fairseq/blob/master/examples/translation/prepare-iwslt14.sh)
!bash fairseq/examples/translation/prepare-iwslt14.sh

## 2. Library reference
1. [tasks](https://fairseq.readthedocs.io/en/latest/tasks.html)
 - translation task와 language modeling task가 있고 나머지 sequence를 다루는 task는 register_task() function decorator를 이용해 등록할 수 있습니다.
2. [models](https://fairseq.readthedocs.io/en/latest/models.html)
 - 모델은 CNN, LSTM, Transformer 기반 모델들이 분류가 되어 있습니다. transformer 모델쪽 코드가 꼼꼼히 잘되어 있습니다. 새로운 모델을 등록하기 위해서는 register_model() function decorator를 이용할 수 있습니다.
3. [criterions](https://fairseq.readthedocs.io/en/latest/criterions.html)
 - 모델 학습을 위한 다양한 loss들이 구현되어 있습니다.
4. [optimizers](https://fairseq.readthedocs.io/en/latest/optim.html)
 - 모델 학습을 위한 다양한 optimizer들이 구현되어 있습니다.
5. l[earning rate schedulers](https://fairseq.readthedocs.io/en/latest/lr_scheduler.html)
 - 모델의 더 나은 학습을 위한 다양한 learning rate scheduler들이 구현되어 있습니다.
6. [data loading and utilities](https://fairseq.readthedocs.io/en/latest/data.html)
 - 전처리 및 데이터 관련 다양한 class들이 구현되어 있습니다.
7. [modules](https://fairseq.readthedocs.io/en/latest/modules.html)
 - 앞의 6군데에 속하지 못한(?) 다양한 모듈들이 구현되어 있습니다.

## 3. [Command-line Tools](https://fairseq.readthedocs.io/en/latest/command_line_tools.html)
fairseq은 학습과 평가를 쉽게할 수 있는 command-line tool을 제공하고 있습니다
각각의 커맨드라인에 대한 설명은 위 링크에 자세히 나와있으니 참고해주시기 바랍니다.
1. fairseq-preprocess
 - 데이터 학습을 위한 vocab을 만들고 data를 구성합니다.
2. fairseq-train
 - 여러 gpu 또는 단일 gpu에서 모델을 학습시킵니다.
3. fairseq-generate
 - 학습된 모델을 이용해 전처리된 데이터를 번역합니다.
4. fairseq-interactive
 - 학습된 모델을 이용해 raw 데이터를 번역합니다.
5. fairseq-score
 - 학습된 모델이 생성한 문장과 정답 문장을 비교해 bleu score를 산출합니다.
6. fairseq-eval-lm
 - language model을 평가할 수 있는 command입니다.


In [ ]:
# 예시 코드를 이용해 직접 전처리부터 평가까지 진행해보겠습니다.
# source-lang: source language
# target-lang: target language
# trainpref: train file prefix
# validpref: valid file prefix
# testpref: test file prefix
# destdir: destination dir
!fairseq-preprocess --source-lang de --target-lang en --trainpref ./iwslt14.tokenized.de-en/train --validpref ./iwslt14.tokenized.de-en/valid --testpref ./iwslt14.tokenized.de-en/test --destdir ./iwslt14.tokenized.de-en/

In [ ]:
# 모델 학습
# (참고: 모델을 동시에 여러개 학습 시키고 싶으신 분들은 노트북 파일을 드라이브에 여러개 복사해서 각 파일마다 실행하면 여러 모델을 동시에 학습시킬 수 있습니다.)
# --arch: architecture
# --optimizer: optimizer {adadelta, adam, adafactor, adagrad, lamb, composite, nag, adamax, sgd}
# --clip-norm: clip threshold of gradients
# --lr: learning rate
# --lr-scheduler: learning rate scheduler {pass_through, cosine, reduce_lr_on_plateau, fixed, triangular, polynomial_decay, tri_stage, manual, inverse_sqrt}
# --criterion loss function {sentence_prediction, ctc, adaptive_loss, label_smoothed_cross_entropy, composite_loss, nat_loss, masked_lm, sentence_ranking, legacy_masked_lm_loss, cross_entropy, model, wav2vec, label_smoothed_cross_entropy_with_alignment, vocab_parallel_cross_entropy}
# --max-tokens: maximum number of tokens in a batch
# --max-epoch: maximum number of training epoch

!fairseq-train ./iwslt14.tokenized.de-en/ --arch transformer_iwslt_de_en --optimizer adam --clip-norm 0.0 --lr 5e-4 --lr-scheduler inverse_sqrt --criterion label_smoothed_cross_entropy --max-tokens 4096 --max-epoch 3

In [ ]:
 # 예측 문장 생성 및 평가
 # checkpoints 폴더에 epoch마다 모델이 저장되고 best checkpoint는 checkpoint_best.pt라는 이름으로 저장됩니다
!fairseq-generate ./iwslt14.tokenized.de-en --path ./checkpoints/checkpoint_best.pt --beam 5 --remove-bpe

In [ ]:
# 여러분만의 모델을 학습해 목표 BLEU score를 달성해주세요 ! 
# command line tools: https://fairseq.readthedocs.io/en/latest/command_line_tools.html
# github: https://github.com/pytorch/fairseq